# TPC4: Ficheiros CSV com listas e funções de agregação

Programa em Python que implementa um conversor de um ficheiro CSV (Comma separated values) para o formato JSON.
Para se poder realizar a conversão pretendida, é importante saber que a primeira linha do CSV dado funciona como cabeçalho que define o que representa cada coluna.


# Bibliotecas importadas:

In [ ]:
import json
import re

# Resolução:

In [ ]:
f = open('./csv/dataset5.csv','r')
f_json = open('./json/dataset5.json','w')

lines = f.readlines()

first_line = lines.pop(0)

header = re.split(r'(?<=[a-zA-Z]),(?=[a-zA-Z])',first_line.strip())

bodyExp = re.compile(r'^(?P<numero>\d+),(?P<nome>(\w+\s*)+),(?P<curso>(\w+\s*)+),*(?P<notas>(\d+|,)+)*$',re.UNICODE)

maxMinExp = re.compile(r'^\w+(\{(?P<nota>\d+)\}|\{(?P<ni>\d+),(?P<ns>\d+)\})')

operatorExp = re.compile(r'(::)(?P<operator>\w+)')


dictjson={}
dictjson['alunos'] = []

for line in lines:
    dictaux={}
    match = bodyExp.search(line.strip())
    dictaux[header[0]] = match.group('numero')
    dictaux[header[1]] = match.group('nome')
    dictaux[header[2]] = match.group('curso')

    if len(header)>3:
        match_nota = maxMinExp.search(header[3])
        lista_notas = []
        if match_nota.group('nota'):
            maximo = int(match_nota.group('nota'))
            notas_group = match.group('notas')
            notas = re.split(',',notas_group)
            for i in range(0,maximo):
                lista_notas.append(notas[i])
        else: 
            minimo = int(match_nota.group('ni'))
            maximo = int(match_nota.group('ns'))
            notas_group = match.group('notas')
            notas = re.split(',',notas_group)
            for i in range(0,maximo):
                if(notas[i].isdigit()):
                    lista_notas.append(notas[i])
        dictaux['Notas'] = lista_notas
        
        match_operator = operatorExp.finditer(header[3])
        if match_operator:
            notas_group = match.group('notas')
            notas = re.split(',',notas_group)
            for match in match_operator:
                op = match['operator']
                if op.lower() == 'sum':
                    valor = 0
                    for nota in notas:
                        if nota.isdigit():
                            valor+= int(nota)
                    dictaux['Notas_sum'] = valor
                elif op.lower() == 'media':
                    valor=0
                    for nota in notas:
                        if nota.isdigit():
                            valor+= int(nota)
                    valor /= len(notas)
                    dictaux['Notas_media'] = valor
                elif op.lower() == 'min':
                    dictaux['Notas_media'] = min(notas)
                elif op.lower() == 'max':
                    dictaux['Notas_media'] = max(notas)

    dictjson['alunos'].append(dictaux)


json.dump(dictjson,f_json,indent=4,ensure_ascii=False)

# Método de resolução:

Para o desenvolvimento desta fase começo por ler as linhas do dataset pretendido, em seguida retiro a primeira linha da lista de linhas resultantes de ler o dataset, esta linha corresponde ao cabeçalho. Realizo então o parse da linha de cabeçalho recorrendo à função 'split' do módulo 're'.
Em seguida realizo o match das restantes linhas do ficheiro com a expressão regular 'bodyExp', e assim recolho o Número, Nome e Curso de cada elemento do ficheiro. Neste momento ocorre a primeira variação, o ficheiro pode ter ou não o campo Notas, e por isso analiso o tamanho do split anteriormente feito, se exister mais de 3 campos então irei realizar um serach com a expressão regular 'minMaxExp' sobre a ultima string proveniente do split. É através desta expressão que descubro se estou perante um valor fixo de notas ou se estou perante um intervalo variável de notas, e assim recolho o número de notas resultante das restantes linhas do ficheiro.
Para verificar se existe ou não mais que uma operação a fazer, utilizando,novamente, a última string proveniente do split do cabeçalho realiza um match com a expressão regular 'operatorExp' que através de um finditer me permite iterar sobre as operações existentes e realizar as mesmas guardando tanto a operação como o seu valor.
Ao longo do program faço uso de um dicionário auxiliar 'dictaux' para guardar todos os parâmetros de cada linha num objeto singular, no final de percorrer a linha adiciono este objeto à lista 'alunos', esta lista corresponde a uma entrada de um dicionário 'dictjson' global utilizado parra fazer a conversão dos dados para o formato json

# Resultados obtidos:

- Exemplo dataset1.csv:

Numero,Nome,Curso
3162,Cândido Faísca,Teatro
7777,Cristiano Ronaldo,Desporto
264,Marcelo Sousa,Ciência Política

Corresponde a uma tabela com 3 registos de informação: a primeira linha de cabeçalho identifica os campos de cada registo, Número, Nome, Curso, e as linhas seguintes contêm os registos de informação.

- Resultado em formato json:
{
    "alunos": [
        {
            "Numero": "3162",
            "Nome": "Cândido Faísca",
            "Curso": "Teatro"
        },
        {
            "Numero": "7777",
            "Nome": "Cristiano Ronaldo",
            "Curso": "Desporto"
        },
        {
            "Numero": "264",
            "Nome": "Marcelo Sousa",
            "Curso": "Ciência Política"
        }
    ]
}

- Exemplo dataset2.csv:

Número,Nome,Curso,Notas{5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,15,16
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,18

No cabeçalho, cada campo poderá ter um número N que representará o número de colunas que esse campo abrange. Por exemplo, imaginemos que ao exemplo anterior se acrescentou um campo Notas, com N = 5.

- Resultado em formato json:
{
    "alunos": [
        {
            "Número": "3162",
            "Nome": "Cândido Faísca",
            "Curso": "Teatro",
            "Notas": [
                "12",
                "13",
                "14",
                "15",
                "16"
            ]
        },
        {
            "Número": "7777",
            "Nome": "Cristiano Ronaldo",
            "Curso": "Desporto",
            "Notas": [
                "17",
                "12",
                "20",
                "11",
                "12"
            ]
        },
        {
            "Número": "264",
            "Nome": "Marcelo Sousa",
            "Curso": "Ciência Política",
            "Notas": [
                "18",
                "19",
                "19",
                "20",
                "18"
            ]
        }
    ]
}


- Exemplo dataset3.csv:

Número,Nome,Curso,Notas{3,5},,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,

Para além de um tamanho único, podemos também definir um intervalo de tamanhos {N, M}, significando que o número de colunas de um certo campo pode ir de N até M

- Resultado em formato json:
{
    "alunos": [
        {
            "Número": "3162",
            "Nome": "Cândido Faísca",
            "Curso": "Teatro",
            "Notas": [
                "12",
                "13",
                "14"
            ]
        },
        {
            "Número": "7777",
            "Nome": "Cristiano Ronaldo",
            "Curso": "Desporto",
            "Notas": [
                "17",
                "12",
                "20",
                "11",
                "12"
            ]
        },
        {
            "Número": "264",
            "Nome": "Marcelo Sousa",
            "Curso": "Ciência Política",
            "Notas": [
                "18",
                "19",
                "19",
                "20"
            ]
        }
    ]
}

- Exemplo dataset4.csv:

Número,Nome,Curso,Notas{3,5}::sum,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,

Para além de listas, podemos ter funções de agregação, aplicadas a essas listas.

- Resultado em formato json:
{
    "alunos": [
        {
            "Número": "3162",
            "Nome": "Cândido Faísca",
            "Curso": "Teatro",
            "Notas": [
                "12",
                "13",
                "14"
            ],
            "Notas_sum": 39
        },
        {
            "Número": "7777",
            "Nome": "Cristiano Ronaldo",
            "Curso": "Desporto",
            "Notas": [
                "17",
                "12",
                "20",
                "11",
                "12"
            ],
            "Notas_sum": 72
        },
        {
            "Número": "264",
            "Nome": "Marcelo Sousa",
            "Curso": "Ciência Política",
            "Notas": [
                "18",
                "19",
                "19",
                "20"
            ],
            "Notas_sum": 76
        }
    ]
}

- Exemplo dataset5.csv:

Número,Nome,Curso,Notas{3,5}::sum::media,,,,,
3162,Cândido Faísca,Teatro,12,13,14,,
7777,Cristiano Ronaldo,Desporto,17,12,20,11,12
264,Marcelo Sousa,Ciência Política,18,19,19,20,

Para além de listas, podemos ter várias funções de agregação, aplicadas a essas listas, seguindo o formato(::operação1::operação2::operação3).

- Resultado em formato json:
{
    "alunos": [
        {
            "Número": "3162",
            "Nome": "Cândido Faísca",
            "Curso": "Teatro",
            "Notas": [
                "12",
                "13",
                "14"
            ],
            "Notas_sum": 39,
            "Notas_media": 7.8
        },
        {
            "Número": "7777",
            "Nome": "Cristiano Ronaldo",
            "Curso": "Desporto",
            "Notas": [
                "17",
                "12",
                "20",
                "11",
                "12"
            ],
            "Notas_sum": 72,
            "Notas_media": 14.4
        },
        {
            "Número": "264",
            "Nome": "Marcelo Sousa",
            "Curso": "Ciência Política",
            "Notas": [
                "18",
                "19",
                "19",
                "20"
            ],
            "Notas_sum": 76,
            "Notas_media": 15.2
        }
    ]
}

Estes são apenas alguns exempos da forma como o cabeçalho do dataset era aceitável, no entanto poderiamos o intervalo de notas, o tipo de operação podia ser de min ou max, e podiamos ter ainda mais operações seguidas no formato (::operação1::operação2::operação3).

Observação: Para objetivos de teste deve correr o programa python 'csvtojson.py', o ficheiro jupiter apenas serve para documentação e análise de resultados.